This notebook is only for testing. Maybe we can build a working model from this, and then we can start putting code into the "model" python module.

In [1]:
import sys
sys.path.append('../py')

In [2]:
import tensorflow as tf
import cavitylearn.data
import numpy as np
import time

In [3]:
config = tf.ConfigProto(
        #device_count = {'GPU': 0}
    )

In [4]:
sess = tf.InteractiveSession(config=config)

In [5]:
with open('../../data/test/test-cavities.filenames.local.txt', 'rt') as cavfile:
    fnames = [r.strip() for r in cavfile]

In [6]:
dataconfig = cavitylearn.data.DataConfig(classes=["SO4","GOL","EDO","HEM","NAG","PO4","ACT"], num_props=4, boxshape=[30, 30, 30])
with open('../../data/test/labels.txt', 'rt') as labelfile:
    trainset = cavitylearn.data.DataSet(labelfile, '/var/cavitylearn', dataconfig)

In [7]:
trainset.N

21687

In [8]:
boxes_placeholder = tf.placeholder(tf.float32, shape=[None, dataconfig.boxshape[0], dataconfig.boxshape[1], dataconfig.boxshape[2], dataconfig.num_props])

In [9]:
labels_placholder = tf.placeholder(tf.float32, shape=[None, dataconfig.num_classes])

In [10]:
NUM_FILTERS=12
PIXELS= dataconfig.boxshape[0] * dataconfig.boxshape[1] * dataconfig.boxshape[2]
BATCHSIZE=100

In [11]:
with tf.variable_scope('conv1') as scope:
    kernel = tf.Variable(tf.truncated_normal([5,5,5,dataconfig.num_props,NUM_FILTERS]))
    conv = tf.nn.conv3d(boxes_placeholder, kernel, [1,1,1,1,1], padding='SAME')
    biases = tf.Variable(tf.zeros([NUM_FILTERS], dtype=tf.float32))
    bias = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relu(bias, name=scope.name)

In [12]:
pool1 = tf.nn.max_pool3d(conv1, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')

In [13]:
norm1 = pool1 # tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta = 0.75, name='norm1')

In [14]:
kernel = tf.Variable(tf.truncated_normal([5,5,5,NUM_FILTERS,NUM_FILTERS]))

In [15]:
kernel.get_shape()

TensorShape([Dimension(5), Dimension(5), Dimension(5), Dimension(12), Dimension(12)])

In [16]:
with tf.variable_scope('conv2') as scope:
    kernel = tf.Variable(tf.truncated_normal([5,5,5,NUM_FILTERS,NUM_FILTERS]))
    conv = tf.nn.conv3d(norm1, kernel, [1,1,1,1,1], padding='SAME')
    biases = tf.Variable(tf.zeros([NUM_FILTERS], dtype=tf.float32))
    bias = tf.nn.bias_add(conv, biases)
    conv2 = tf.nn.relu(bias, name=scope.name)

In [17]:
norm2 = conv2

In [18]:
pool2 = tf.nn.max_pool3d(norm2, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')

In [19]:
with tf.variable_scope('local3') as scope:
    reshape = tf.reshape(pool2, [BATCHSIZE, -1])
    dim = reshape.get_shape()[1].value
    print(dim)

None


In [20]:
reshape.get_shape()

TensorShape([Dimension(100), Dimension(None)])

In [21]:
pool2.get_shape()

TensorShape([Dimension(None), Dimension(8), Dimension(8), Dimension(8), Dimension(12)])

In [22]:
FC_SIZE = 384 # number of neurons per fully connected layer

In [23]:
with tf.variable_scope('local3') as scope:
    dim = np.prod(pool2.get_shape().as_list()[1:])
    pool2_flat = tf.reshape(pool2, [-1, dim])
    weights = tf.Variable(tf.truncated_normal([dim, FC_SIZE]), dtype=tf.float32)
    biases = tf.Variable([FC_SIZE], dtype=tf.float32)
    local3 = tf.nn.relu(tf.matmul(pool2_flat, weights) + biases, name=scope.name)

In [24]:
with tf.variable_scope('local3') as scope:
    dim = np.prod(local3.get_shape().as_list()[1:])
    local3_flat = tf.reshape(local3, [-1, dim])
    weights = tf.Variable(tf.truncated_normal([dim, int(FC_SIZE/2)]), dtype=tf.float32)
    biases = tf.Variable([int(FC_SIZE/2)], dtype=tf.float32)
    local4 = tf.nn.relu(tf.matmul(local3_flat, weights) + biases, name=scope.name)

In [25]:
with tf.variable_scope('softmax_linear') as scope:
    dim = np.prod(local4.get_shape().as_list()[1:])
    weights = tf.Variable(tf.truncated_normal([dim, dataconfig.num_classes]), dtype=tf.float32)
    biases = tf.Variable([dataconfig.num_classes], dtype=tf.float32)
    softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)

In [26]:
logits = softmax_linear

In [27]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(labels_placholder * tf.log(logits), reduction_indices=[1]))

In [28]:
sess.run(tf.initialize_all_variables())

In [29]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [30]:
correct_prediction = tf.equal(tf.argmax(logits,1),  tf.argmax(labels_placholder,1))

In [31]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [32]:
NUM_BATCHES=30

In [33]:
start_time = time.time()
for i in range(NUM_BATCHES):
    labels, boxes = trainset.next_batch(BATCHSIZE)
    train_step.run(feed_dict={boxes_placeholder: boxes, labels_placholder: labels})
    print(';', str(accuracy.eval(feed_dict={boxes_placeholder: boxes, labels_placholder: labels})), end='')
end_time = time.time()

; 0.26; 0.24; 0.26; 0.23; 0.26; 0.24; 0.21; 0.26; 0.25; 0.26; 0.29; 0.33; 0.24; 0.23; 0.25; 0.24; 0.23; 0.23; 0.22; 0.28; 0.27; 0.2; 0.24; 0.19; 0.28; 0.27; 0.29; 0.23; 0.33; 0.31

In [34]:
print("Total time: ", str(end_time - start_time))
print("Time per batch: ", str((end_time - start_time)/NUM_BATCHES))

Total time:  27.6542489528656
Time per batch:  0.9218082984288534
